#  Clean  data  scraped from Avito: Cars
- Data was scraped from avito, Region - entire Russia, any car brand, price 450 000 -1m
- Date November 2021

In [1]:
import pandas as pd
import re
import numpy as np
from copy import deepcopy

In [2]:
df = pd.read_csv('scraped_avito_cars_450-1m.csv')
df.head(1)

,car_brand,car_price,car_location,car_details,ad_date,car_dealer,car_description,today,car_url
0,"Mitsubishi Lancer, 2014",650 000 ₽,Москва,"100 181 км, 1.8 CVT (140 л.с.), седан, передни...",Несколько секунд назад,NaN,Прoдаю надёжный автомобиль в достойнойном тeхн...,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...


In [3]:
colnames = ['car','price','location','details','date','dealer','inform','today','url']
df.columns = colnames

In [4]:
df.head(1)

,car,price,location,details,date,dealer,inform,today,url
0,"Mitsubishi Lancer, 2014",650 000 ₽,Москва,"100 181 км, 1.8 CVT (140 л.с.), седан, передни...",Несколько секунд назад,NaN,Прoдаю надёжный автомобиль в достойнойном тeхн...,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...


In [5]:
df.shape

(5656, 9)

In [6]:
df.describe()

,car,price,location,details,date,dealer,inform,today,url
count,5656,5656,5656,5656,5656,1663,5047,5656,5656
unique,1586,576,718,4511,87,377,3934,1,4665
top,"Hyundai Solaris, 2014",750 000 ₽,Москва,"4 км, 1.5 MT (106 л.с.), седан, передний, бензин",1 час назад,АвтоЛидер Варшавка,Комплект Зимней (Летней) Резины По Сезону В По...,2021-11-23,https://www.avito.ru//moskva/avtomobili/hyunda...
freq,54,126,916,17,1198,94,56,5656,6


In [7]:
df.price.describe()

count          5656
unique          576
top       750 000 ₽
freq            126
Name: price, dtype: object

# format columns, clear special characters, split

In [8]:
df.price = df.price.replace({'от|₽|без|скидки|\s':''}, regex=True).copy()

In [9]:
sum(df.price.str.contains('₽'))

0

In [10]:
## prototype split details
b =  pd.DataFrame(df.details.apply(lambda x: str(x).split(',')).to_list()) 
b.head(2)
#df.loc[:, 2].unique()

,0,1,2,3,4
0,100 181 км,1.8 CVT (140 л.с.),седан,передний,бензин
1,213 000 км,1.6 MT (105 л.с.),седан,передний,бензин


In [11]:
df[['mileage', 'box_engine', 'body', 'drive', 'fuel']] = df.details.str.split(',', expand = True)

In [12]:
df.head(1)

,car,price,location,details,date,dealer,inform,today,url,mileage,box_engine,body,drive,fuel
0,"Mitsubishi Lancer, 2014",650000,Москва,"100 181 км, 1.8 CVT (140 л.с.), седан, передни...",Несколько секунд назад,NaN,Прoдаю надёжный автомобиль в достойнойном тeхн...,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...,100 181 км,1.8 CVT (140 л.с.),седан,передний,бензин


In [13]:
df.body.unique()

array([' седан', ' хетчбэк', ' фургон', ' внедорожник', ' универсал',
       ' минивэн', ' пикап', ' микроавтобус', ' купе', ' кабриолет'],
      dtype=object)

In [14]:
## replace the first bracket with , to split the column later
## ) is a special character, so escape it with \
df.box_engine = df.box_engine.replace({'\(':','}, regex=True).copy()

In [15]:
df[['vol_box', 'hp']] = df.box_engine.str.split(',', expand = True)

In [16]:
df.head(1)

,car,price,location,details,date,dealer,inform,today,url,mileage,box_engine,body,drive,fuel,vol_box,hp
0,"Mitsubishi Lancer, 2014",650000,Москва,"100 181 км, 1.8 CVT (140 л.с.), седан, передни...",Несколько секунд назад,NaN,Прoдаю надёжный автомобиль в достойнойном тeхн...,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...,100 181 км,"1.8 CVT ,140 л.с.)",седан,передний,бензин,1.8 CVT,140 л.с.)


In [17]:
df.hp = df.hp.replace({'л.с.\)':''}, regex=True).copy()
df.hp = df.hp.replace({'\s':''}, regex=True).copy()

### vol_box : Etxract vol and hp : Regex Method

In [18]:
df['vol'] = df['vol_box'].str.extract(r'(\d+.\d+)').astype('float')

In [19]:
df.iloc[[228,229]]

,car,price,location,details,date,dealer,inform,today,url,mileage,box_engine,body,drive,fuel,vol_box,hp,vol
228,"LADA Largus, 2020",985000,"Саратовская область, Энгельсский р-н, Энгельс","47 499 км, 1.6 MT (106 л.с.), универсал, перед...",9 минут назад,NaN,Состояние нового автомобиля! Один хозяин! Беза...,2021-11-23,https://www.avito.ru//engels/avtomobili/lada_l...,47 499 км,"1.6 MT ,106 л.с.)",универсал,передний,бензин,1.6 MT,106,1.6
229,"Mercedes-Benz M-класс AMG, 2007",800000,"Курская область, Курск","285 000 км, AT (510 л.с.), внедорожник, полный...",9 минут назад,NaN,Авто полностью обслужен не каких вложений не т...,2021-11-23,https://www.avito.ru//kursk/avtomobili/mercede...,285 000 км,"AT ,510 л.с.)",внедорожник,полный,бензин,AT,510,NaN


In [20]:
df['box'] = df['vol_box'].str.extract(r'([a-zA-Z]+)')

In [21]:
df.iloc[[228,229]]

,car,price,location,details,date,dealer,inform,today,url,mileage,box_engine,body,drive,fuel,vol_box,hp,vol,box
228,"LADA Largus, 2020",985000,"Саратовская область, Энгельсский р-н, Энгельс","47 499 км, 1.6 MT (106 л.с.), универсал, перед...",9 минут назад,NaN,Состояние нового автомобиля! Один хозяин! Беза...,2021-11-23,https://www.avito.ru//engels/avtomobili/lada_l...,47 499 км,"1.6 MT ,106 л.с.)",универсал,передний,бензин,1.6 MT,106,1.6,MT
229,"Mercedes-Benz M-класс AMG, 2007",800000,"Курская область, Курск","285 000 км, AT (510 л.с.), внедорожник, полный...",9 минут назад,NaN,Авто полностью обслужен не каких вложений не т...,2021-11-23,https://www.avito.ru//kursk/avtomobili/mercede...,285 000 км,"AT ,510 л.с.)",внедорожник,полный,бензин,AT,510,NaN,AT


### vol_box : Etxract vol and hp : Split, expand method (longer, inefficnet)

In [22]:
## values in vol_box col with no engine vol data are placed in 
## a column for volumne after split
cc = df.vol_box.str.split(' ', expand = True).copy()
cc.iloc[[227, 228,229]]

,0,1,2,3
227,,1.6,MT,
228,,1.6,MT,
229,,AT,,None


In [23]:
## Move during split method
## the 4 in lambda Basically regualtes how far to the right the Dummy 0 will be pushed 
a = pd.DataFrame(df.vol_box.apply(lambda x: str(x).split(' ')).
                                          apply(lambda x: [np.nan]*(4-len(x)) + x).to_list())
a.iloc[[227, 228,229]]

,0,1,2,3
227,,1.6,MT,
228,,1.6,MT,
229,NaN,,AT,


In [24]:
str ([np.nan]*(4 - len(' AT')) ) + ' AT'

'[nan] AT'

In [25]:
df.head(2)

,car,price,location,details,date,dealer,inform,today,url,mileage,box_engine,body,drive,fuel,vol_box,hp,vol,box
0,"Mitsubishi Lancer, 2014",650000,Москва,"100 181 км, 1.8 CVT (140 л.с.), седан, передни...",Несколько секунд назад,NaN,Прoдаю надёжный автомобиль в достойнойном тeхн...,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...,100 181 км,"1.8 CVT ,140 л.с.)",седан,передний,бензин,1.8 CVT,140,1.8,CVT
1,"Mazda 3, 2008",490000,"Свердловская область, Екатеринбург","213 000 км, 1.6 MT (105 л.с.), седан, передний...",Несколько секунд назад,NaN,"По кузову имеется один окрас, остальное все в ...",2021-11-23,https://www.avito.ru//ekaterinburg/avtomobili/...,213 000 км,"1.6 MT ,105 л.с.)",седан,передний,бензин,1.6 MT,105,1.6,MT


### rearrange columns once more

In [26]:
df =df.drop(['details', 'box_engine', 'vol_box'], axis =1)
df=df[['car','price','mileage','hp', 'vol','box',
       'body', 'drive', 'fuel', 'location',  'inform','date','today','url']]

In [27]:
df.iloc[[228,229]]

,car,price,mileage,hp,vol,box,body,drive,fuel,location,inform,date,today,url
228,"LADA Largus, 2020",985000,47 499 км,106,1.6,MT,универсал,передний,бензин,"Саратовская область, Энгельсский р-н, Энгельс",Состояние нового автомобиля! Один хозяин! Беза...,9 минут назад,2021-11-23,https://www.avito.ru//engels/avtomobili/lada_l...
229,"Mercedes-Benz M-класс AMG, 2007",800000,285 000 км,510,NaN,AT,внедорожник,полный,бензин,"Курская область, Курск",Авто полностью обслужен не каких вложений не т...,9 минут назад,2021-11-23,https://www.avito.ru//kursk/avtomobili/mercede...


In [28]:
df.mileage = df.mileage.replace({'км':''}, regex = True)
df.mileage = df.mileage.replace({'\s':''}, regex = True)

In [29]:
## split car
dd = df.car.str.split(',', expand = True).copy()
dd.head(3)

,0,1
0,Mitsubishi Lancer,2014
1,Mazda 3,2008
2,ВАЗ (LADA) Granta,2020


In [30]:
dd.loc[:, 0].describe()

count                5656
unique                495
top       Hyundai Solaris
freq                  294
Name: 0, dtype: object

In [31]:
df[['car2', 'year']] = df.car.str.split(',', expand = True).copy()
df = df.drop(['car'], axis = 1)
df.rename(columns = {'car2':'car'}, inplace = True) ## rename just a once column
df=df[['car','year','price','mileage','hp', 'vol','box',
       'body', 'drive', 'fuel', 'location',  'inform','date','today','url']]

In [32]:
df.head(1)

,car,year,price,mileage,hp,vol,box,body,drive,fuel,location,inform,date,today,url
0,Mitsubishi Lancer,2014,650000,100181,140,1.8,CVT,седан,передний,бензин,Москва,Прoдаю надёжный автомобиль в достойнойном тeхн...,Несколько секунд назад,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...


## make relevant columns numeric

In [33]:
## try without errors = 'coerce', to see what data aberrations you might have
## in this case, ValueError: Unable to parse string "AT" at position 13
## investigate and see where 'AT' is
cols = ['year','price','mileage','hp', 'vol']
df[cols] = df[cols].apply(pd.to_numeric) 

### vody, drive fuel, location - change to English (optional)

In [34]:
df.drive.unique()

array([' передний', ' полный', ' задний'], dtype=object)

In [35]:
# strip whitespace
df.body = df.body.replace({'\s':''}, regex = True)
df.drive = df.drive.replace({'\s':''}, regex = True)
df.fuel = df.fuel.replace({'\s':''}, regex = True)

In [36]:
df.drive.unique()

array(['передний', 'полный', 'задний'], dtype=object)

In [37]:
df.body.unique()

array(['седан', 'хетчбэк', 'фургон', 'внедорожник', 'универсал',
       'минивэн', 'пикап', 'микроавтобус', 'купе', 'кабриолет'],
      dtype=object)

In [42]:
df.fuel.unique()

array(['бензин', 'дизель', 'электро', 'гибрид', 'газ'], dtype=object)

In [39]:
df = df.replace({'body' : { 'седан' : 'sedan', 'хетчбэк' : 'hatchback', 'фургон' : 'van',
                          'внедорожник':'SUV', 'универсал':'station_vagon','минивэн':'mini_van',
                          'пикап':'pickup', 'микроавтобус':'minibus','купе':'coupe', 'кабриолет':'convertible'}})
df = df.replace({'drive' : { 'передний' : 'front_wheel', 'полный' : 'all_wheel', 'задний' : 'rear_wheel' }})
df = df.replace({'fuel' : { 'бензин':'gasoline', 'дизель':'diesel', 'электро':'electric',
                           'гибрид':'hybrid','газ':'natural_gas'}})

In [40]:
df.head()

,car,year,price,mileage,hp,vol,box,body,drive,fuel,location,inform,date,today,url
0,Mitsubishi Lancer,2014,650000,100181,140,1.8,CVT,sedan,front_wheel,gasoline,Москва,Прoдаю надёжный автомобиль в достойнойном тeхн...,Несколько секунд назад,2021-11-23,https://www.avito.ru//moskva/avtomobili/mitsub...
1,Mazda 3,2008,490000,213000,105,1.6,MT,sedan,front_wheel,gasoline,"Свердловская область, Екатеринбург","По кузову имеется один окрас, остальное все в ...",Несколько секунд назад,2021-11-23,https://www.avito.ru//ekaterinburg/avtomobili/...
2,ВАЗ (LADA) Granta,2020,742000,25600,106,1.6,MT,sedan,front_wheel,gasoline,"Свердловская область, Екатеринбург",= Самый большой Автомолл! Более 700 автомобиле...,Несколько секунд назад,2021-11-23,https://www.avito.ru//ekaterinburg/avtomobili/...
3,Toyota Tank,2018,590000,43000,69,1.0,CVT,hatchback,all_wheel,gasoline,"Приморский край, Владивостокский г.о., Владиво...",Автомобиль под заказ из Японии. \n\nПробег ука...,Несколько секунд назад,2021-11-23,https://www.avito.ru//vladivostok/avtomobili/t...
4,Kia Rio,2018,999000,65000,123,1.6,AT,sedan,front_wheel,gasoline,"Астраханская область, Астрахань","Автомобиль в отличном состоянии, вложений не т...",Несколько секунд назад,2021-11-23,https://www.avito.ru//astrahan/avtomobili/kia_...


### save results

In [41]:
df.to_csv('cleaned_avito_cars_450-1m.csv')